Package manager:
Conda

Environment: Python 3.10

Packages:
Pyarrow, Ydata-synthetic, jupyter, pandas

In [ ]:
import pandas as pd

Reading original data

In [ ]:
original_data = pd.read_parquet('/home/waqas/Thesis/SyntheticDataGeneration/data/test_107_reduced.parquet')

Defining relevant column types

In [ ]:
failure_columns = [
	'cd_counter_rollers_installation',
	'ld_counter_rollers_installation',
	'closing_device_failure',
	'misalignment_cd_vs_ld',
	'misalignment_cd_vs_ld_sill_gap',
	'belt_tension',
	'pulley_is_touching_belt',
	'zero_position',
]

environment_columns = [
	'cmcouplerfriction',
	'cmdoorfriction',
	'cmelectronicage',
	'cmvibration',
]

static_state_columns = [
	'doorcyclecounter',
]

time_state_columns = [
	'doorforce',
	'doorspeed',
	'doorposition',
	# 'time',
]

Organizing the data based on failure types

In [ ]:
failure_groups = original_data.groupby(failure_columns)

In [ ]:
failure_groups_meta = pd.DataFrame(
	[group_values for group_values, _ in failure_groups],
	columns = failure_columns,
)
failure_groups_meta.to_parquet('data/preprocessed/group_meta.parquet')
failure_groups_meta

Saving the data in separate files based on failure types

In [ ]:
def save_groups():
	ref_values = list(failure_groups_meta.itertuples(index=False, name=None))

	for (group_index, (values, group)) in enumerate(failure_groups):
		assert(ref_values[group_index] == values)
		g = group[environment_columns + static_state_columns + time_state_columns]
		g_exploded = g.explode(time_state_columns)

		# testing the accuracy by comparing the expected results to the actual result
		expected_num_rows = sum([len(group.iloc[j]['doorforce']) for j in range(group.shape[0])])
		assert(expected_num_rows == g_exploded.shape[0])

		g_exploded.to_parquet(f'data/preprocessed/group_{group_index}.parquet')

save_groups()